In [ ]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(temperature = 0, model="gemma3:12b")

response = llm.invoke("겨울철에 내한성이 강한 나무에는 어떤 것이 있을까요?")
print(response)

In [ ]:
# PDF 문서 로드 및 텍스트 추출 

from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("수목관리업무매뉴얼(표지제외).pdf", extract_images=True)
pages = loader.load() 
pages[9].page_content

In [ ]:
#문장 임베딩 및 벡터 저장소 생성 
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

#문서를 문장으로 분리 
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, 
    chunk_overlap=100,
)

docs = text_splitter.split_documents(pages)

#문장을 임베딩으로 변환하고 벡터 저장소에 저장 
embeddings = HuggingFaceEmbeddings(
    model_name = 'jhgan/ko-sroberta-nli',
    model_kwargs = {'device': 'cpu'},
    encode_kwargs = {'normalize_embeddings':True},
)

vectorstore = Chroma.from_documents(docs, embeddings)

In [ ]:
#검색 쿼리 
query = "겨울철에 내한성이 강한 나무에는 어떤 것이 있을까요?"

#가장 유사도가 높은 문장을 하나만 추출 
retriever = vectorstore.as_retriever(search_kwargs={'k':5})

docs = retriever.invoke(query)

print(len(docs))
print(docs[0].page_content)
print(docs[0].metadata)

In [ ]:
print(docs[1].page_content)
print(docs[1].metadata)

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import OllamaLLM

# LLM
model = OllamaLLM(
    model="gemma3:12b",
    temperature=0
)

#prompt
template = '''Answer the question based only on the following context in Korean :
{context}

Question: {question}'''

prompt = PromptTemplate.from_template(template)

def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

#Rag chain 연결 
rag_chain = ({'context': retriever | format_docs, 'question' : RunnablePassthrough()}
             | prompt
             | model 
             | StrOutputParser())

# Chain 실행 
query = "겨울철에 내한성이 강한 나무에는 어떤 것이 있을까요?"
rag_chain.invoke(query)



In [ ]:
query = "겨울철에 추위에 약한 나무에는 어떤 것이 있을까요?"
rag_chain.invoke(query)